In [8]:
import cv2
import matplotlib.pyplot as plt
import os
import paddle as paddle
import paddle.fluid as fluid
import PIL.Image as Image
import random
from fourcrop import htgCrap
import numpy as np
import time




#预测模型
class inferModel:
    def __init__(self,model_name):
        self.math_map = ['0','1','2','3','4','5','6','7','8','9','(',')','+','-','*','/','=','a','t','s','n','sqrt','.','PI','y']
        model_net_path = 'models/infer_model/'+str(model_name)+'/'
        place = fluid.CPUPlace()
        exe = fluid.Executor(place)
        [infer_program, feeded_var_names, target_var] = fluid.io.load_inference_model(dirname=model_net_path, executor=exe)
        self.args = [exe,infer_program,feeded_var_names,target_var]
    
    def infer_char(self,image):
        exe, infer_program , feeded_var_names ,target_var = self.args
        data_shape = np.array(image).shape[0]
        image = np.array(image).reshape(-1,1,data_shape,data_shape).astype('float32')
        image = (image) /255.0
        result = exe.run(program = infer_program,
                    feed = {feeded_var_names[0]:image},
                     fetch_list = target_var
                    )
        result_char = self.math_map[np.argmax(result)]
        return result_char


#连通域图形处理
class ConnDomain:
    def __init__(self):
        pass
        
    def cropCharContours(self,formula):#提取图片中的轮廓，解决父子边框嵌入问题，父子合为一体
        try:
            contours,hierarchy= cv2.findContours(formula, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
        except Exception:
            _,contours,hierarchy= cv2.findContours(formula, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
        x_lenth = formula.shape[1]
        y_lenth = formula.shape[0]
        temp_image = np.zeros((y_lenth,x_lenth))
        #父轮廓与子轮廓结合
        temp_contours = [[] * 1 for i in range(len(hierarchy[0]))]
        total_len = np.sum(np.array([len(contour) for contour in contours]))
        total_avg = total_len/len(hierarchy[0])
        total_avg /= 8
        for index in range(len(hierarchy[0])):
            father = hierarchy[0][index][3]
            if len(contours[index]) < total_avg:#过滤过小的噪点
                hierarchy[0][index][3] = 0
            for cont1 in contours[index]:#对Contours数据组进行整合处理
                temp_contours[index].append([cont1[0][0],cont1[0][1]])
            if father!=-1:#将子轮廓并入父轮廓数组中
                for cont1 in contours[index]:
                    temp_contours[father].append([cont1[0][0],cont1[0][1]])
                temp_contours[index] = []
        deal_contours = []
        for index in range(len(hierarchy[0])):#将所有的父轮廓提取出来
            father = hierarchy[0][index][3]
            if father == -1:
                deal_contours.append(temp_contours[index])
        return deal_contours
    
    def getCharLocation(self,formulas):#根据Char的位置进行组排序
        formula = cv2.bitwise_not(formulas)
        mg = cv2.blur(formula,(2,2))#羽化
        ret, formula = cv2.threshold(mg, 10, 255,cv2.THRESH_BINARY)
        x_lenth = formula.shape[1]
        y_lenth = formula.shape[0]
        contours = self.cropCharContours(formula)
        charLocation = []
        for contour in contours:
            x_min = x_lenth
            y_min = y_lenth
            x_max = 0
            y_max = 0
            for x,y in contour:
                if x < x_min:
                    x_min = x
                if y < y_min:
                    y_min = y
                if x > x_max:
                    x_max = x
                if y > y_max:
                    y_max = y
            after_contour = np.array(contour)-[x_min,y_min]
            charLocation.append([[x_min,y_min,x_max-x_min,y_max-y_min],after_contour])
        def pxCulate(com):
            x_min = com[0][0]
            y_min = com[0][1]
            return x_min
        charLocation.sort(key=pxCulate)
        return charLocation
    
    def cropChar(self,formulas):#提取字符的起始xy轴坐标以及宽高以及对字符图片进行微处理（填充边框）
        formula = self.getCharLocation(formulas)
        result_image_list = []
        locas = formula
        result_image = []
        for loca in locas:
            after_contour = loca[1]
            x_min,y_min,x_lenght,y_lenght = loca[0]
            lable_image = np.array(Image.new('L',(x_lenght+10,y_lenght+10)))#生成一个空图
            for index in after_contour:#将边框嵌入空图，并且缩放左右5px，上下5px
                x = index[0]+5
                y = index[1]+5
                lable_image[y][x]=255
            mask = np.zeros([lable_image.shape[0]+2, lable_image.shape[1]+2], np.uint8)
            copy_image = lable_image
            cv2.floodFill(copy_image, mask,(0,0), (99 , 0, 0), cv2.FLOODFILL_MASK_ONLY)#将背景进行灰化处理
            def find_point(read_image):#找到字符内部的像素作为洪水填充的种子结点
                y = int(read_image.shape[0]/2)
                light_flage = False
                frist_falge = False
                black_flage = False
                find_x = -1
                find_y = -1
                for i in range(int(read_image.shape[1])):
                    if light_flage == False and read_image[y][i] == 255:
                        light_flage = True
                    elif light_flage == True and frist_falge == False and read_image[y][i] == 0:
                        black_flage = True
                        frist_falge = True
                        find_x = i
                        find_y = y
                    elif light_flage == True and black_flage == True and read_image[y][i] == 255:
                        return find_x,find_y
                return 0,0
            cv2.floodFill(copy_image, mask,find_point(copy_image), (255 , 255, 255), cv2.FLOODFILL_MASK_ONLY)#洪水填充
            _,copy_image = cv2.threshold(copy_image,127,255,cv2.THRESH_BINARY)#临时背景替换成二值化图
            h,w = copy_image.shape
            #以下为将字符缩放成64*64画幅
            data_shape = 50
            printBack = Image.new('L',(64,64))
            if w>h:
                h = int(data_shape*(h/w))
                w = data_shape
                copy_image = Image.fromarray(copy_image).resize((w,h))
            else:
                w = int(data_shape*(w/h))
                h = data_shape
                copy_image = Image.fromarray(copy_image).resize((w,h))
            printBack.paste(copy_image,(int((64-w)/2),int((64-h)/2)))
            #缩放完之后将字符进行加粗
            mg = cv2.blur(np.array(printBack),(3,3))#羽化
            ret, printBack = cv2.threshold(mg, 10, 255,cv2.THRESH_BINARY)
            result_image.append([loca[0],printBack])
        return result_image
    
    def getChar(self,image):
        htgCrapAPI = htgCrap(image)
        formulas = htgCrapAPI.crapFormula()#提取图片中的式子
        char_list = self.cropChar(formulas)
        return char_list


#容错排序算法
class charFix:
    def __init__(self,char_list,infer_list):
        self.char_list = char_list
        self.infer_list = infer_list
        
    def child_belong_in(self,father_position,child_position):#子数字是否在父数字里面
        fx,fy,fw,fh = father_position
        cx,cy,cw,ch = child_position
        if cx>fx and cx+cw<fx+fw:#判断是否在里面，比如分数
            return True
        else:
            return False

    def child_belong_position(self,father_position,child_position):#子数字在父数字的什么位置，1上面，2中间，3下面,4在外面
        fx,fy,fw,fh = father_position
        cx,cy,cw,ch = child_position
        if self.child_belong_in(father_position,child_position):#如果在父数字里面
            if cy+ch < fy+0.1*fh:#在上面
                return 1
            elif cy > fy+0.6*fh:#在下面
                return 3
            elif cx+cw<fx+0.5*fw and cy+ch<fy+0.7*fh:#针对根号
                return 1
            else:
                return 2
        else:#除此之外还有在右边，比如平方和小数点，以size做标准,特殊操作
            cs = cw*ch
            fs = fw*fh
            if cx < fx+(fw*2-(cy+ch-fy)):
                print(child_position)
                if cs < fs * 0.3:#小于部分体积
                    if cy+ch < fy+0.3*fh:#在上面
                        return 1
                    elif cy > fy+0.7*fh:#在下面
                        return 3
                    else:
                        return 4
                elif cs < fs:#若小于大体积 针对于°
                    if cy+ch < fy:#全在上面则返回上面
                        return 1
                    elif cy > fy+fh:
                        return 3
                    else:
                        return 4
                elif cx < fx+fw:
                    if cy+ch < fy:#全在上面则返回上面
                        return 1
                    elif cy > fy + fh:
                        return 3
                    else:
                        return 4 
                elif cx+cw<fx+0.5*fw and cy+ch<fy+0.5*fh:#针对根号
                    return 1
                else:
                    return 4
            else:
                return 4

    def iter_fill(self):#将字符用矩阵排布
        char_index = 0
        re_arr = []
        char_list = self.infer_list
        result_char = self.char_list
        while char_index<len(result_char):
            if char_index == 0:
                re_arr.append([char_list[char_index],[],[],[],result_char[char_index][0]])
            else:
                fcp = re_arr[len(re_arr)-1][4]
                tcp = result_char[char_index][0]
                fc = re_arr[len(re_arr)-1][0]
                tc = char_list[char_index]
                if fc == 'sq' and tc == '-':#sqrt减号
                    cbp = 4
                else:
                    cbp = self.child_belong_position(fcp,tcp)
                itarr = re_arr
                while cbp != 4:
                    itarr = itarr[len(itarr)-1][cbp]
                    if len(itarr) != 0:
                        fcp = itarr[len(itarr)-1][4]
                        cbp = self.child_belong_position(fcp,tcp)
                    else:
                        break
                itarr.append([char_list[char_index],[],[],[],result_char[char_index][0]])
            char_index+=1
        return re_arr

    def judge_minuts_sign(self,ar):#判断减号 1减号 2除号 3分数 4等号
        lu = len(ar[1])#上面次数
        lm = len(ar[2])#中间的次数
        ld = len(ar[3])#下面次数
        if lu==0 and ld==0:
            return 1
        elif (lu==0 and ld==1 and lm==0 and ar[3][0][0] in ['-']) or (lu==1 and ld==0 and lm==0 and ar[1][0][0] in ['-']):
            return 4
        elif lu==1 and ld==1:
            lun = ar[1][0][0]
            ldn = ar[3][0][0]
            keyarr = ['0','1','.','(',')']
            if lun in keyarr and ldn in keyarr:
                return 2
            else:
                return 3
        elif lm==1 and ld==1:
            lun = ar[2][0][0]
            ldn = ar[3][0][0]
            keyarr = ['0','1','.','(',')']
            if (lun in keyarr) and (ldn in keyarr):
                return 2
            else:
                return 3
        elif lu==1 and lm==1:
            lun = ar[1][0][0]
            ldn = ar[2][0][0]
            keyarr = ['0','1','.','(',')']
            if (lun in keyarr) and (ldn in keyarr):
                return 2
            else:
                return 3
        else:
            return 3

    def judge_normal_char(self,ar):#判断普通符号，0无,1小数点、2次幂、3都有
        lu = len(ar[1])
        ld = len(ar[3])
        if lu==0 and ld==0:
            return 0
        elif lu>0 and ld==0:
            return 2
        elif ld>0 and lu==0:
            return 1
        else:
            return 3

    def judge_tragle_sign(self,ar):#返回具体的字符和迭代次数
        fc = ar[0][0]
        le = len(ar)
        if fc=='s' or fc == '5':
            if le>=3 and ar[2][0]=='n':
                return 'sin',3
            elif le>=4 and ar[3][0]=='n':
                return 'sin',4
            elif le>=2 and ar[1][0]=='-' and fc=='s':
                return '5',2
            else:
                return '5',1
        elif fc=='(' and le>=3:
            if ar[1][0] == '0' and (ar[2][0] == 's' or ar[2][0]=='5'):
                return 'cos',3
            elif len(ar[0][1])>=1 and ar[0][1][0][0] == '-':
                return '5',1
            elif len(ar[0][2])>=1 and ar[0][2][0][0] == '-':
                return '5',1
            else:
                return '(',1
        elif fc=='t' and le>=3:
            if ar[1][0] == 'a' and ar[2][0] == 'n':
                return 'tan',3
            else:
                return 't',1
        else:
            return fc,1

    def disposeChildText(self,array):
        ct = ''
        aid = 0
        while aid<len(array):
            ar = array[aid]
            fc = ar[0]
            aidn = 1
            if fc == '-':#减号特殊，除号and分数and减号
                ms = self.judge_minuts_sign(ar)
                if ms==1:
                    ct+='-'
                elif ms==2:
                    ct+='/'
                elif ms==4:
                    ct+='='
                elif ms==3:
                    ctup = self.disposeChildText(ar[1])
                    ctdown = self.disposeChildText(ar[3])
                    ct+='(('+ctup+')/('+ctdown+'))'
            elif fc=='s' or fc=='(' or fc=='t' or fc == '5':#处理sin cos tan
                cts , aidn = self.judge_tragle_sign(array[aid:])
                if cts=='5' and aidn==1 and len(ar[1])>1:
                    ct+='5^'+self.disposeChildText(ar[1][1:])
                else:
                    ct+=cts
            elif fc=='Pi':#π的左半边
                ct+='PI'
                aidn=2
            elif fc=='sqrt':#根号
                sqn = '2'
                if len(ar[1])>0:
                    sqn = self.disposeChildText(ar[1])
                sqi = self.disposeChildText(ar[2])
                ct+='sqrt'+sqn+'('+sqi+')'
            elif fc=='sq':#分开的根号
                sqn = '2'
                if len(ar[1]) > 0 :
                    sqn = self.disposeChildText(ar[1])
                sqi = self.disposeChildText(array[aid+1][3])
                ct+='sqrt'+sqn+'('+sqi+')'
                aidn=2
            else:#处理正常数字
                jnc = self.judge_normal_char(ar)
                if jnc==0:
                    ct+=fc
                elif jnc==1:
                    ct+=fc+'.'
                elif jnc==2 or jnc==3:
                    ctup = self.disposeChildText(ar[1])
                    glarr = ['+','-','*','/','(',')','s','t','y','a']
                    if ctup not in glarr:
                        ct+=fc+'^('+ctup+')'
                    if jnc==3:
                        ct+='.'
            aid+=aidn
        return ct
    
    def getText(self):
        return self.disposeChildText(self.iter_fill())
    
    #四则汇总API
class four_operations:
    def __init__(self,model_name):
        #初始化工具类
        self.infer_model = inferModel(model_name)
        self.dnn = ConnDomain()
    
    def infer_image(self,image):
        image = np.array(Image.open(image))
        char_image = self.dnn.getChar(image)
        formula_image_list = []
        for char in char_image:
            result = self.infer_model.infer_char(char[1])
            formula_image_list.append(result)
        charProgram = charFix(char_image,formula_image_list)
        print(charProgram.iter_fill())
        result = charProgram.getText()
        return result

In [9]:
test = four_operations(3)

In [10]:
test.infer_image('test_image/71.jpg')

[855, 173, 23, 26]
[1182, 127, 37, 33]
[1512, 79, 57, 195]
[1587, 50, 48, 36]
[['-', [['(', [], [], [], [189, 139, 56, 191]], ['6', [], [], [], [293, 180, 33, 89]], ['0', [], [], [], [347, 217, 22, 35]], ['0', [], [], [], [374, 213, 24, 33]], ['+', [], [], [], [421, 212, 60, 57]], ['s', [], [], [], [513, 204, 49, 70]], ['1', [], [], [], [575, 230, 16, 38]], ['n', [], [], [], [609, 203, 40, 65]], ['(', [], [], [], [682, 169, 33, 107]], ['7', [], [], [], [754, 176, 35, 106]], ['0', [['0', [], [], [], [855, 173, 23, 26]]], [], [], [805, 216, 37, 39]], [')', [], [], [], [886, 159, 30, 126]], ['-', [], [], [], [959, 215, 59, 12]], ['7', [], [], [], [1070, 155, 44, 110]], ['0', [['2', [], [], [], [1182, 127, 37, 33]]], [], [], [1129, 198, 39, 39]], ['+', [], [], [], [1234, 187, 65, 58]], ['sqrt', [], [['7', [], [], [], [1408, 178, 34, 67]], ['5', [], [], [], [1456, 168, 45, 73]]], [], [1342, 131, 184, 133]], [')', [['2', [], [], [], [1587, 50, 48, 36]]], [], [], [1512, 79, 57, 195]]], [], [[

'(((600+sin(70^(0))-70^(2)+sqrt2(75))^(2))/(256*374))'